In [1]:
from ews.models import *
import pandas as pd
import geojson
import numpy as np

In [8]:
d = pd.read_csv('D:/DWC_plattform/riverflow_locations.csv')

In [9]:
d

,SiteName,SiteReferencementName,SiteType,SourceID,SiteLatitude(y),SiteLongitude(x)
0,Alfortville,alfortville_seine,Riverflow,NaN,48.797558,2.418510
1,Austerllitz,austerlitz,Riverflow,NaN,48.845091,2.366067
2,Creteil,cr�teil_marne,Riverflow,NaN,48.790437,2.472380
3,Gournay-sur-Marne,gournay_sur_marne_marne,Riverflow,NaN,48.865347,2.574696
4,Saint-Fargeau-Ponthierry,saint_fargeau_ponthierry_marne,Riverflow,NaN,48.793223,2.739247


In [10]:
for index, row in d.iterrows():
    loc = Site()
    loc.name = row['SiteName']
    loc.ref_name = row['SiteReferencementName']
    loc.geom = {"coordinates": [row['SiteLongitude(x)'],row['SiteLatitude(y)']], "type": "Point"}
    loc.feature_type = FeatureType.objects.get(name = row['SiteType'])
    loc.owner = User.objects.get(id = 1)
    try:
        loc.save()
    except:
        print(loc)

In [18]:
rf = pd.read_csv('D:/DigitalWaterCity/data_processed/river_flows.csv', parse_dates = True, index_col = "Date")

In [19]:
rf

,Q (m3/s),name,river
Date,,,
2016-01-01 00:00:00+00:00,90.0,Alfortville,Seine
2016-01-01 01:00:00+00:00,90.3,Alfortville,Seine
2016-01-01 02:00:00+00:00,86.3,Alfortville,Seine
2016-01-01 03:00:00+00:00,88.6,Alfortville,Seine
2016-01-01 04:00:00+00:00,90.3,Alfortville,Seine
...,...,...,...
2020-09-14 06:00:00+00:00,103.0,Austerlitz,Seine
2020-09-14 07:00:00+00:00,105.0,Austerlitz,Seine
2020-09-14 08:00:00+00:00,111.0,Austerlitz,Seine


In [21]:
rf = rf.groupby("name").resample("D").mean().reset_index()

In [25]:
rf["Q (m3/s)"] = rf["Q (m3/s)"].round(2)

array(['Alfortville', 'Austerlitz', 'Creteil', 'Gournay-sur-Marne',
       'Saint-Fargeau-Ponthierry'], dtype=object)

In [27]:
for index, row in rf.iterrows():
    dpoint = FeatureData()
    dpoint.date = row["Date"]
    dpoint.value = row["Q (m3/s)"]
    dpoint.site = Site.objects.get(name = row['name'])
    dpoint.save()
    

DoesNotExist: Site matching query does not exist.